Use BERT trained on tweets. 

Gives 80% accuracy.

(implementation of this was based on a [rumour dection task](https://github.com/Bachfischer/COMP90042-Rumour-Detection-on-Twitter))

In [1]:
# Import libraries
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# Check connection to gpu
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Aug 11 10:11:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip3 install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 197 kB 26.7 MB/s 
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=255b3729a2a6db30f24fca00c8b05d8d7404b9b5cbee5b30c38e852a9cb6959c
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe7452ac7d8d83b1930f8a6205c3c9649b24e80f9029fc38
Successfully built emoji


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 29.1 MB/s 
     |████████████████████████████████| 596 kB 63.6 MB/s 
     |████████████████████████████████| 6.6 MB 61.2 MB/s 
     |████████████████████████████████| 101 kB 5.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Importing necessary packages
import pandas as pd
import numpy as np
import os
from tqdm._tqdm_notebook import tqdm_notebook
import re
import torch
from transformers import get_linear_schedule_with_warmup,AdamW,AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import TensorDataset,DataLoader, RandomSampler, SequentialSampler, Dataset
import time
import datetime 
from sklearn.metrics import confusion_matrix, classification_report

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  """


In [ ]:
# Getting training / testing datasets may need to change put in own training / testing sets once all annotations completed

master_df = pd.read_csv('/content/drive/MyDrive/Diss_data/ROBpopulist_NN_trainingdata.csv')[['text', 'label']]

master_df

,text,label
0,"The DUP are devout leavers, while Sinn Fein, d...",1
1,Grt to have DEFRA SecState @andrealeadsom in #...,0
2,Well let's trust voters in Richmond Park give ...,0
3,Speechless! Sweden declares flying the Isis fl...,0
4,As @allianceparty Infrastructure spokesperson ...,0
...,...,...
2948,#Labour are not funded by billionaires but ord...,1
2949,I’ve just listened to Dominic Raab refusing to...,1
2950,🚨Polls are open🚨\n\nVote 🌹 Labour 🌹 today for ...,1
2951,The county hasn't raised the local taxes to th...,1


In [ ]:
politweets_17 = pd.read_csv('/content/drive/MyDrive/Diss_data/Politicians_Tweets_2017.csv')
politweets_17

,conversation_id,username,create_at,text,retweet_count,reply_count,like_count,hashtags
0,877624914987163651,SKinnock,2017-06-21T20:30:59.000Z,It is vital that we secure a transition deal a...,7,7,5,NaN
1,877540748652150785,SKinnock,2017-06-21T14:56:33.000Z,We need full public inquiry into the scandal o...,37,4,30,#bleedingtruth #QueensSpeech
2,877209374296616960,SKinnock,2017-06-20T16:59:47.000Z,Thanks - much appreciated! https://t.co/ir3Pg3...,5,0,3,NaN
3,877151434495733760,SKinnock,2017-06-20T13:09:33.000Z,We need breathing space of a #TransitionDeal p...,5,0,4,#TransitionDeal #EEA
4,877146946703687680,SKinnock,2017-06-20T12:51:43.000Z,A transition deal is vital to ensure we don't ...,5,0,3,#EEA
...,...,...,...,...,...,...,...,...
229893,861217616232411137,jamesblanchard,2017-05-07T13:54:15.000Z,An elected representative really ought to be a...,3,1,5,NaN
229894,859512676204109828,jamesblanchard,2017-05-02T20:59:25.000Z,Robert Peston says this letter is worth readin...,11,0,10,NaN
229895,859512062657081347,jamesblanchard,2017-05-02T20:56:59.000Z,Robert Peston says this letter is worth readin...,1,0,2,NaN
229896,854466902927843328,jamesblanchard,2017-04-18T22:49:19.000Z,The woman who says she can get the best deal o...,0,0,2,NaN


In [ ]:
politweets_19 = pd.read_csv('/content/drive/MyDrive/Diss_data/Politicians_Tweets_2019.csv')
politweets_19


,conversation_id,username,create_at,text,retweet_count,reply_count,like_count,hashtags
0,1209156155349622787,SKinnock,2019-12-23T16:57:47.000Z,Contrary to views of some London-based lib-lef...,3,3,35,NaN
1,1208081373224996869,SKinnock,2019-12-20T17:46:59.000Z,I was paired &amp; slipped today. If I’d been ...,7,23,37,NaN
2,1207687004630028288,SKinnock,2019-12-19T15:39:54.000Z,Very pleased that after a gruelling 2 1/2 year...,4,1,16,NaN
3,1207645319804133376,SKinnock,2019-12-19T12:54:15.000Z,Choice was always clear: soft Brexit + real ch...,59,501,237,NaN
4,1207641465175891969,SKinnock,2019-12-19T12:38:56.000Z,"""Messages about fast-paced economic change/glo...",2,25,37,#WakeUpAndSmellTheCoffee
...,...,...,...,...,...,...,...,...
260131,1185090605128605696,AnnaPerrett,2019-10-18T07:09:52.000Z,Love our NHS? Boris Johnson doesn’t care. He w...,4,0,3,#ProtectOurNHS
260132,1184395982697648128,AnnaPerrett,2019-10-16T09:09:41.000Z,We need some 21st century solutions to transpo...,1,1,5,#York
260133,1184161559439912960,AnnaPerrett,2019-10-15T17:38:11.000Z,As the daughter of a postie (of 28 years) some...,0,0,7,NaN
260134,1184006207750713344,AnnaPerrett,2019-10-15T07:20:52.000Z,Callous but sadly no longer shocking story of ...,0,0,1,NaN


In [ ]:
# Split into train / test data sets
from sklearn.model_selection import train_test_split

training, testing = train_test_split(master_df, test_size=0.3, random_state=0)

In [ ]:
# Shuffling the training data 
from sklearn.utils import shuffle

#training = shuffle(training)

In [ ]:
training

,text,label
2676,Despite what people say about me in this campa...,1
1610,"See, I think the BBC needs to think really car...",1
2287,TIL many things.,0
6,"As Mayor of London, Boris Johnson closed 10 fi...",1
1403,Brilliant response from residents in Braunston...,0
...,...,...
763,Remembering this Polish pilot who risked his l...,0
835,#Bracknell #Sandhurst #Wokingham #Crowthorne ...,0
1653,Busy campaigning for #GE2017 with @HaringeyLib...,0
2607,"Amazing, thank you so much. Yes, very sadly #l...",0


In [ ]:
tqdm_notebook.pandas()

#for preprocessing, get rid of non-words and links
def preprocess(df):
    symbols_regular_expression = re.compile('[^0-9a-z #+_]')
    link_regular_expression = re.compile('https\S+|www\S+|http\S+')

    df['text'] = df.text.progress_apply(lambda x: str(x).lower())
    df['text'] = df.text.progress_apply(lambda x: link_regular_expression.sub(' ', x))
    df['text'] = df.text.progress_apply(lambda x: symbols_regular_expression.sub(' ', x))
    
    return df

In [ ]:
# Pre-processing training / testing dataframes
train_df = preprocess(training)

  0%|          | 0/2067 [00:00<?, ?it/s]

  0%|          | 0/2067 [00:00<?, ?it/s]

  0%|          | 0/2067 [00:00<?, ?it/s]

In [ ]:
# Change df here to whatever you want predictions on
test_df = preprocess(politweets_19)

  0%|          | 0/260136 [00:00<?, ?it/s]

  0%|          | 0/260136 [00:00<?, ?it/s]

  0%|          | 0/260136 [00:00<?, ?it/s]

In [ ]:
#use bert tokenizer to encode the data in order to pass it to the model
def bert_encode(df, tokenizer):
    input_ids = []
    attention_masks = []
    for sent in df[["text"]].values:
        sent = sent.item()
        encoded_dict = tokenizer.encode_plus(
                            sent,                      
                            add_special_tokens = True, 
                            max_length = 65,           
                            pad_to_max_length = True,
                            truncation = True,
                            return_attention_mask = True,   
                            return_tensors = 'pt',    
                    )
           
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    inputs = {
    'input_word_ids': input_ids,
    'input_mask': attention_masks
    }
    return inputs

In [ ]:
#put dataset into dataloader in order to support batching and memeory pinning
def prepare_dataloaders(train_df,test_df,batch_size=8):
    tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)
    
    tweet_train = bert_encode(train_df, tokenizer)
    tweet_train_labels = train_df.label.astype(int)
    
    tweet_test = bert_encode(test_df, tokenizer)

    input_ids, attention_masks = tweet_train.values()

    print(input_ids.shape)
    print(attention_masks.shape)
    labels = torch.tensor(tweet_train_labels.values)
    train_dataset = TensorDataset(input_ids, attention_masks, labels)



    
    input_ids, attention_masks = tweet_test.values()
    test_dataset = TensorDataset(input_ids, attention_masks)

    
    train_dataloader = DataLoader(
                train_dataset,
                sampler = RandomSampler(train_dataset), 
                batch_size = batch_size 
            )


    test_dataloader = DataLoader(
                test_dataset, 
                sampler = SequentialSampler(test_dataset), 
                batch_size = batch_size
            )
    return train_dataloader, test_dataloader

In [ ]:
# CHANGE THIS TO CHANGE THE DATA IT WORKS ON
train_dataloader,test_dataloader = prepare_dataloaders(train_df, test_df)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


torch.Size([2067, 65])
torch.Size([2067, 65])


In [ ]:
#make load bertweet as a model for sequence classification, with adam optimizer 
def prepare_model(model_class="vinai/bertweet-base",num_classes=2,model_to_load=None,total_steps=-1):
    model = AutoModelForSequenceClassification.from_pretrained(
        model_class,
        num_labels = num_classes,  
        output_attentions = False, 
        output_hidden_states = False,
    )
 

    optimizer = AdamW(model.parameters(),
                    lr = 5e-5,
                    eps = 1e-8
                    )
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, 
                                                num_training_steps = total_steps)

    if model_to_load is not None:
        try:
            model.roberta.load_state_dict(torch.load(model_to_load))
            print("LOADED MODEL")
        except:
            pass
    return model, optimizer, scheduler

In [ ]:
epochs = 8
total_steps = len(train_dataloader) * epochs

modelv2, optimizer, scheduler = prepare_model("vinai/bertweet-base" ,num_classes=2, model_to_load=None, total_steps = total_steps)

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: 

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

def train(model,optimizer,scheduler,train_dataloader,epochs):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    training_stats = []
    total_t0 = time.time()

    for epoch_i in range(0, epochs):

        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')
        
        t0 = time.time()
        total_train_loss = 0
        model.train()
        for step, batch in enumerate(train_dataloader):
            if step % 40 == 0 and not step == 0:
                elapsed = format_time(time.time() - t0)
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            model.zero_grad()        
            outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask, 
                                labels=b_labels)
            loss = outputs.loss
            logits = outputs.logits
            total_train_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
        avg_train_loss = total_train_loss / len(train_dataloader)            
        training_time = format_time(time.time() - t0)

  
        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epoch took: {:}".format(training_time))

    print("")
    print("Training complete!")

    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [ ]:
# Computationally intensive - reduce epoch numbers? 
train(modelv2,optimizer,scheduler,train_dataloader, epochs)


======== Epoch 1 / 8 ========
Training...
  Batch    40  of    259.    Elapsed: 0:00:06.
  Batch    80  of    259.    Elapsed: 0:00:11.
  Batch   120  of    259.    Elapsed: 0:00:16.
  Batch   160  of    259.    Elapsed: 0:00:22.
  Batch   200  of    259.    Elapsed: 0:00:27.
  Batch   240  of    259.    Elapsed: 0:00:33.

  Average training loss: 0.51
  Training epoch took: 0:00:36

======== Epoch 2 / 8 ========
Training...
  Batch    40  of    259.    Elapsed: 0:00:06.
  Batch    80  of    259.    Elapsed: 0:00:11.
  Batch   120  of    259.    Elapsed: 0:00:17.
  Batch   160  of    259.    Elapsed: 0:00:22.
  Batch   200  of    259.    Elapsed: 0:00:28.
  Batch   240  of    259.    Elapsed: 0:00:34.

  Average training loss: 0.39
  Training epoch took: 0:00:36

======== Epoch 3 / 8 ========
Training...
  Batch    40  of    259.    Elapsed: 0:00:06.
  Batch    80  of    259.    Elapsed: 0:00:11.
  Batch   120  of    259.    Elapsed: 0:00:17.
  Batch   160  of    259.    Elapsed: 0:00

In [ ]:
def predict(model,test_dataloader):
    model.eval()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    preds = []

    for batch in test_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        with torch.no_grad():        
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
            logits = outputs.logits

        logits = logits.detach().cpu().numpy()
        for logit in logits:
            preds.append(logit)

    return preds

In [ ]:
# VERY COMPUTATIONALLY INTENSIVE WITH 2017 TWEETS 16 minutes! 
# WRITE THIS 3 TIMES, ONCE FOR TEST RESULTS, ONCE FOR 17 AND 19
result = predict(modelv2,test_dataloader)

In [ ]:
result

[array([-2.1468558,  2.1399753], dtype=float32),
 array([ 2.5728223, -2.119202 ], dtype=float32),
 array([0.1991226 , 0.14833026], dtype=float32),
 array([-3.0226295,  2.9763997], dtype=float32),
 array([-3.0443842,  2.9478185], dtype=float32),
 array([-3.0683494,  2.987688 ], dtype=float32),
 array([ 3.8284438, -3.5803962], dtype=float32),
 array([-2.9607248,  2.8988247], dtype=float32),
 array([-3.0905383,  3.0127795], dtype=float32),
 array([-2.7373004,  2.705694 ], dtype=float32),
 array([-3.0426035,  2.9460511], dtype=float32),
 array([ 3.67429  , -3.3854494], dtype=float32),
 array([-3.061915 ,  2.9671497], dtype=float32),
 array([ 3.7229707, -3.3846872], dtype=float32),
 array([ 3.7229707, -3.3846872], dtype=float32),
 array([-3.02825  ,  2.9394464], dtype=float32),
 array([-2.9466677,  2.8648105], dtype=float32),
 array([ 3.8823552, -3.5694366], dtype=float32),
 array([ 3.8083742, -3.5080864], dtype=float32),
 array([ 3.845446 , -3.5886612], dtype=float32),
 array([ 3.670259 , 

In [ ]:
pred_labels = np.argmax(result, axis = 1)

In [ ]:
confusion_matrix(test_df.label, pred_labels)

array([[560, 115],
       [ 69, 142]])

In [ ]:
print(classification_report(test_df.label, pred_labels, target_names=['not-populist', 'populist']))

              precision    recall  f1-score   support

not-populist       0.89      0.83      0.86       675
    populist       0.55      0.67      0.61       211

    accuracy                           0.79       886
   macro avg       0.72      0.75      0.73       886
weighted avg       0.81      0.79      0.80       886



In [ ]:
testing['model_lab'] = pred_labels
testing

,text,label,model_lab
1301,a huge happy #europeday to everyone this elec...,1,1
2942,no child in the uk today should face homelessn...,1,1
2286,this afternoon busy bunch working with our tea...,0,0
2669,as a candidate in the 2019 general election i...,0,0
1934,can we check on this royal_greenwich tfl thi...,0,0
...,...,...,...
2476,#esherandwalton residents have told me our #nh...,1,1
2316,set this speech in stone,0,0
715,i was in beautiful sandbach this week amp me...,1,1
988,i am the one and only candidate to respond ...,0,0


In [ ]:
politweets_17['populist'] = pred_labels
politweets_17

,conversation_id,username,create_at,text,retweet_count,reply_count,like_count,hashtags,model_lab,populist
0,877624914987163651,SKinnock,2017-06-21T20:30:59.000Z,it is vital that we secure a transition deal a...,7,7,5,NaN,0,0
1,877540748652150785,SKinnock,2017-06-21T14:56:33.000Z,we need full public inquiry into the scandal o...,37,4,30,#bleedingtruth #QueensSpeech,1,1
2,877209374296616960,SKinnock,2017-06-20T16:59:47.000Z,thanks much appreciated,5,0,3,NaN,0,0
3,877151434495733760,SKinnock,2017-06-20T13:09:33.000Z,we need breathing space of a #transitiondeal p...,5,0,4,#TransitionDeal #EEA,0,0
4,877146946703687680,SKinnock,2017-06-20T12:51:43.000Z,a transition deal is vital to ensure we don t ...,5,0,3,#EEA,0,0
...,...,...,...,...,...,...,...,...,...,...
229893,861217616232411137,jamesblanchard,2017-05-07T13:54:15.000Z,an elected representative really ought to be a...,3,1,5,NaN,0,0
229894,859512676204109828,jamesblanchard,2017-05-02T20:59:25.000Z,robert peston says this letter is worth readin...,11,0,10,NaN,0,0
229895,859512062657081347,jamesblanchard,2017-05-02T20:56:59.000Z,robert peston says this letter is worth readin...,1,0,2,NaN,0,0
229896,854466902927843328,jamesblanchard,2017-04-18T22:49:19.000Z,the woman who says she can get the best deal o...,0,0,2,NaN,1,1


In [ ]:
politweets_19['populist'] = pred_labels
politweets_19

,conversation_id,username,create_at,text,retweet_count,reply_count,like_count,hashtags,populist
0,1209156155349622787,SKinnock,2019-12-23T16:57:47.000Z,contrary to views of some london based lib lef...,3,3,35,NaN,1
1,1208081373224996869,SKinnock,2019-12-20T17:46:59.000Z,i was paired amp slipped today if i d been ...,7,23,37,NaN,0
2,1207687004630028288,SKinnock,2019-12-19T15:39:54.000Z,very pleased that after a gruelling 2 1 2 year...,4,1,16,NaN,0
3,1207645319804133376,SKinnock,2019-12-19T12:54:15.000Z,choice was always clear soft brexit + real ch...,59,501,237,NaN,1
4,1207641465175891969,SKinnock,2019-12-19T12:38:56.000Z,messages about fast paced economic change glo...,2,25,37,#WakeUpAndSmellTheCoffee,1
...,...,...,...,...,...,...,...,...,...
260131,1185090605128605696,AnnaPerrett,2019-10-18T07:09:52.000Z,love our nhs boris johnson doesn t care he w...,4,0,3,#ProtectOurNHS,1
260132,1184395982697648128,AnnaPerrett,2019-10-16T09:09:41.000Z,we need some 21st century solutions to transpo...,1,1,5,#York,1
260133,1184161559439912960,AnnaPerrett,2019-10-15T17:38:11.000Z,as the daughter of a postie of 28 years some...,0,0,7,NaN,0
260134,1184006207750713344,AnnaPerrett,2019-10-15T07:20:52.000Z,callous but sadly no longer shocking story of ...,0,0,1,NaN,1


In [ ]:
# Save results to csv
testing.to_csv('retrain_test_w_labels.csv')
!cp retrain_test_w_labels.csv "drive/My Drive/Diss_data"

In [ ]:
# Save results to csv
politweets_17.to_csv('LBLD_politician_2017_tweets.csv')
!cp LBLD_politician_2017_tweets.csv "drive/My Drive/Diss_data"

In [ ]:
# Save results to csv
politweets_19.to_csv('LBLD_politician_2019_tweets.csv')
!cp LBLD_politician_2019_tweets.csv "drive/My Drive/Diss_data"

In [ ]:
# After re-training, still achieve accuracy of 81%!!